In [1]:
import numpy as np
import pandas as pd
import matplotlib
import sklearn

In [2]:
import json

In [3]:
def parse(path):
    with open(path, 'r') as f:
        text_data = f.read()
    for l in text_data.split('\n'):
        yield l

In [4]:
def getDF(path):
    dicts = []
    for d in parse(path):
        #print(d)
        try:
            dicts.append(json.loads(d))
        except:
            print(d)
            print(type(d))
    return pd.DataFrame(dicts)

In [5]:
data = getDF('Data/Movies_and_TV_5.json')


<class 'str'>


In [6]:
data.reviewerID.nunique()

123960

In [7]:
data['isGood'] = data['overall'] > 3

In [8]:
data.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime', 'isGood'],
      dtype='object')

## Первые шаги

Будем обучать модель с помощью стохастического градиента. BFGS на данных упорно не хочет сходиться даже при увеличении верхней границы числа итераций, в то время как SAG как раз рекомендован в документации для разреженных данных и данных большого размера.

In [51]:
from sklearn.model_selection import train_test_split

In [10]:
[train_data, test_data, train_ans, test_ans] = train_test_split(data['reviewText'], data['isGood'],train_size=0.8, test_size=0.2, shuffle=True, random_state=24)

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [43]:
vectorizer = TfidfVectorizer()

In [44]:
transformed_train = vectorizer.fit_transform(train_data)

In [45]:
transformed_train[0]

<1x712164 sparse matrix of type '<class 'numpy.float64'>'
	with 184 stored elements in Compressed Sparse Row format>

In [53]:
from sklearn.linear_model import LogisticRegression

In [46]:
clf_first = LogisticRegression(solver='sag', random_state=25)

In [47]:
clf_first.fit(transformed_train, train_ans)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=25, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

In [48]:
transformed_test = vectorizer.transform(test_data)

In [54]:
from sklearn.metrics import f1_score

In [49]:
f1_score(test_ans, clf_first.predict(transformed_test))

0.9196446836742547

Ого!

Подберем лучший параметр регуляризации с помощью кросс-валидации

In [17]:
from sklearn.linear_model import LogisticRegressionCV

In [18]:
clfCv = LogisticRegressionCV(Cs=[1.0,1.2,1.4,1.6], cv=5, scoring='f1', random_state=25, solver='sag')

In [19]:
clfCv.fit(transformed_train, train_ans)

LogisticRegressionCV(Cs=[1.0, 1.2, 1.4, 1.6], class_weight=None, cv=5,
                     dual=False, fit_intercept=True, intercept_scaling=1.0,
                     l1_ratios=None, max_iter=100, multi_class='auto',
                     n_jobs=None, penalty='l2', random_state=25, refit=True,
                     scoring='f1', solver='sag', tol=0.0001, verbose=0)

In [20]:
clfCv.C_

array([1.2])

Таким образом, классификатор достигает наибольшего успеха при параметре регуляризации C, равном 1.2, будем использовать его и далее.

In [24]:
f1_score(test_ans, clfCv.predict(transformed_test))

0.9196452546328897

Стало не сильно лучше, но в любом случае, мы получили какой-то просто потрясающий результат. Посмотрим, что будет, если поиграться с параметрами tfidf.

## n-грамы

### биграм

In [25]:
vectorizer_2 = TfidfVectorizer(ngram_range=(2,2))

In [27]:
transformed_train_2 = vectorizer_2.fit_transform(train_data)

In [32]:
clf = LogisticRegression(C=1.2, random_state=25, solver='sag')

In [34]:
clf.fit(transformed_train_2, train_ans)

LogisticRegression(C=1.2, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=25, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

In [35]:
transformed_test_2 = vectorizer_2.transform(test_data)

In [36]:
f1_score(test_ans, clf.predict(transformed_test_2))

0.9295234169328924

Небольшой, но прогресс!

### 3-грам

In [13]:
vectorizer_3 = TfidfVectorizer(ngram_range=(3,3))

In [ ]:
transformed_train_3 = vectorizer_3.fit_transform(train_data)

In [ ]:
clf_3 = LogisticRegression(C=1.2, random_state=25, solver='sag')

Что-то kernel прилег(((

### (1,2) - грам

In [14]:
vectorizer_12 = TfidfVectorizer(ngram_range=(1,2))

In [15]:
transformed_train_12 = vectorizer_12.fit_transform(train_data)

In [19]:
clf_12 = LogisticRegression(C=1.2, random_state=25, solver='sag')

In [21]:
clf_12.fit(transformed_train_12, train_ans)

LogisticRegression(C=1.2, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=25, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
transformed_test_12 = vectorizer_12.transform(test_data)

In [24]:
f1_score(test_ans, clf_12.predict(transformed_test_12))

0.9325761802089773

Еще круче!!!

## Редкие и частые слова

### max_df

In [32]:
for max_df in np.arange(0.9, 1.01, 0.02):
    print(f"max_df = {max_df}")
    vectorizer_max_df = TfidfVectorizer(max_df=max_df)
    transformed_train_max_df = vectorizer_max_df.fit_transform(train_data)
    clf_max_df = LogisticRegression(C=1.2, random_state=25, solver='sag')
    clf_max_df.fit(transformed_train_max_df, train_ans)
    transformed_test_max_df = vectorizer_max_df.transform(test_data)
    print(f1_score(test_ans, clf_max_df.predict(transformed_test_max_df)))

max_df = 0.9
0.9196452546328897
max_df = 0.92
0.9196452546328897
max_df = 0.9400000000000001
0.9196452546328897
max_df = 0.9600000000000001
0.9196452546328897
max_df = 0.9800000000000001
0.9196452546328897
max_df = 1.0
0.9196452546328897


Вывод: в данном случае никакие ограничения на максимальную частоту не улучшают качество модели. 

In [33]:
for min_df in np.arange(0.00, 0.06, 0.01):
    print(f"min_df = {min_df}")
    vectorizer_min_df = TfidfVectorizer(min_df=min_df)
    transformed_train_min_df = vectorizer_min_df.fit_transform(train_data)
    clf_min_df = LogisticRegression(C=1.2, random_state=25, solver='sag')
    clf_min_df.fit(transformed_train_min_df, train_ans)
    transformed_test_min_df = vectorizer_min_df.transform(test_data)
    print(f1_score(test_ans, clf_min_df.predict(transformed_test_min_df)))

min_df = 0.0
0.9196452546328897
min_df = 0.01
0.9050043721369029
min_df = 0.02
0.8959838277432098
min_df = 0.03
0.8875914291504688
min_df = 0.04
0.8852227163106108
min_df = 0.05
0.8809551645525224


In [73]:
for min_df in np.arange(1, 10, 1):
    print(f"min_df = {min_df}")
    vectorizer_min_df = TfidfVectorizer(min_df=min_df)
    transformed_train_min_df = vectorizer_min_df.fit_transform(train_data)
    clf_min_df = LogisticRegression(C=1.2, random_state=25, solver='sag')
    clf_min_df.fit(transformed_train_min_df, train_ans)
    transformed_test_min_df = vectorizer_min_df.transform(test_data)
    print(f1_score(test_ans, clf_min_df.predict(transformed_test_min_df)))

min_df = 1
0.9196452546328897
min_df = 2
0.9196261612057766
min_df = 3
0.9196449357534976
min_df = 4
0.9196522274602128
min_df = 5
0.9196713555083249
min_df = 6
0.9196620352985355
min_df = 7
0.9196284071750158
min_df = 8
0.9196031602980052
min_df = 9
0.9195919800383388


Получаем, что отбрасывание слов, встречающихся менее чем 5 документах дает небольшую прибавку к качеству

## Нормализация

Попробуем сначала еще немного поковыряться в аргументах tfidf. Кстати, в нем по умолчанию все слова приводятся к нижнему регистру.

Попробуем удалить стоп-слова:

In [60]:
vectorizer_normalizer_2 = TfidfVectorizer(analyzer='word', stop_words='english')

In [61]:
transformed_train_norm = vectorizer_normalizer_2.fit_transform(train_data)

In [70]:
clf_for_norm = LogisticRegression(C=1.2, random_state=25, solver='sag')

In [71]:
clf_for_norm.fit(transformed_train_norm, train_ans)

LogisticRegression(C=1.2, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=25, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

In [64]:
transformed_test_norm = vectorizer_normalizer_2.transform(test_data)

In [72]:
f1_score(test_ans, clf_for_norm.predict(transformed_test_norm))

0.9163479905649444

Удаление "стоп-слов" будто бы даже немного ухудшило качество.

Ну раз так, попробуем сначала вручную преобразовать входные тексты: удалим лишние пробельные символы, выкинем пунктуацию и попробуем выкинуть числа.

In [111]:
import re

In [150]:
def my_normalizer(string):
    strr = string.lower() #нижний регист
    strr = re.sub('\W', ' ',strr) #выкидываем не буквы и не цифры
    strr = re.sub('\s+', ' ', strr) #заменяем последовательности из пробельных символов на обычные пробелы
    strr = strr.strip() #выкидываем пробельные символы в начале и конце текста
    return strr

In [151]:
my_normalizer('SDAS\n\n {}-==\n*7&65612  \t\tvcxvxc !!!!,,, s,s,s,s,')

'sdas 7 65612 vcxvxc s s s s'

In [157]:
[norm_train_data, norm_test_data, train_ans, test_ans] = train_test_split(data['reviewText'].apply(my_normalizer), data['isGood'],train_size=0.8, test_size=0.2, shuffle=True, random_state=24)

In [164]:
vectorizer = TfidfVectorizer(min_df=5)

In [165]:
transformed_norm_train = vectorizer.fit_transform(norm_train_data)

In [166]:
clf_for_my_norm = LogisticRegression(C=1.2, random_state=25, solver='sag')

In [167]:
clf_for_my_norm.fit(transformed_norm_train, train_ans)

LogisticRegression(C=1.2, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=25, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

In [168]:
transformed_norm_test = vectorizer.transform(norm_test_data)

In [169]:
f1_score(test_ans, clf_for_my_norm.predict(transformed_norm_test))

0.9196713555083249

Это не дает ощутимого прироста к качеству, что довольно как по мне станно. Наверное, это связано с тем, что в tfidf и так по умолчанию встроена определенная предобработка текста.

Выплоним "удаление акцентов и нормализацию символов на этапе предварительной обработки" (дословный перевод документации):

In [50]:
vectorizer_normalizer_1 = TfidfVectorizer(strip_accents='unicode')

In [51]:
transormed_train_norm = vectorizer_normalizer_1.fit_transform(train_data)

In [52]:
clf_for_norm = LogisticRegression(C=1.2, random_state=25, solver='sag')

In [54]:
clf_for_norm.fit(transormed_train_norm, train_ans)

LogisticRegression(C=1.2, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=25, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

In [57]:
transformed_test_norm = vectorizer_normalizer_1.transform(test_data)

In [58]:
f1_score(test_ans, clf_for_norm.predict(transformed_test_norm))

0.9196452546328897

Никакого профита(

Попробуем теперь воспользоваться некоторыми инструментами библиотеки NLTK

In [170]:
import nltk

#### Стемминг

Тут есть два встроенных алгоритма: более универсальный (общий) алгоритм Портера и более агрессивный "Снежный шарик". Опробуем первый (так как стемминг всех текстов что-то очень затратным по времени выходит). Воспользуемся также встроенным токенизатором для разбиения текста на слова (токены). 

In [9]:
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize

In [21]:
stemmer = PorterStemmer()

Пример

In [16]:
data.loc[578937]['reviewText'][:170]

'A sinister mad scientist, Dr. Goldfoot (Vincent Price), along with his incompetent assistant Igor, is looking to conquer the world by amassing the wealth of all the world'

In [18]:
def porter_stemmer_func(string):
    stemmed_text = ''
    for x in word_tokenize(string):
        stemmed_text = stemmed_text + (stemmer.stem(x) + ' ')
    return stemmed_text

In [22]:
porter_stemmer_func(data.loc[578937]['reviewText'])[:160]

'A sinist mad scientist , dr. goldfoot ( vincent price ) , along with hi incompet assist igor , is look to conquer the world by amass the wealth of all the world'

In [210]:
[train_stem_data, test_stem_data, train_ans, test_ans] = train_test_split(data['reviewText'].apply(stemmer_func), data['isGood'],train_size=0.8, test_size=0.2, shuffle=True, random_state=24)
vectorizer = TfidfVectorizer(min_df=5)
transformed_stem_train = vectorizer.fit_transform(train_stem_data)
clf_for_stem = LogisticRegression(C=1.2, random_state=25, solver='sag')
clf_for_stem.fit(transformed_stem_train, train_ans)
transformed_stem_test = vectorizer.transform(test_stem_data)
print(f1_score(test_ans, clf_for_stem.predict(transformed_stem_test)))

0.9162888886389118


Мда, эта штука обучалась триллион лет и на выходе дала чуть ли не ухудшение качества)

#### Лемматизация

In [35]:
from nltk.stem import WordNetLemmatizer

In [41]:
lemmatizer = WordNetLemmatizer()

In [43]:
def lemmer_func(string):
    lemmed_text = ''
    for x in word_tokenize(string):
        lemmed_text = lemmed_text + (lemmatizer.lemmatize(x) + ' ')
    return lemmed_text

In [47]:
data.loc[578937]['reviewText'][:170]

'A sinister mad scientist, Dr. Goldfoot (Vincent Price), along with his incompetent assistant Igor, is looking to conquer the world by amassing the wealth of all the world'

In [50]:
lemmer_func(data.loc[578937]['reviewText'])[:175]

'A sinister mad scientist , Dr. Goldfoot ( Vincent Price ) , along with his incompetent assistant Igor , is looking to conquer the world by amassing the wealth of all the world'

In [55]:
[train_lem_data, test_lem_data, train_ans, test_ans] = train_test_split(data['reviewText'].apply(lemmer_func), data['isGood'],train_size=0.8, test_size=0.2, shuffle=True, random_state=24)
vectorizer = TfidfVectorizer(min_df=5)
transformed_lem_train = vectorizer.fit_transform(train_lem_data)
clf_for_lem = LogisticRegression(C=1.2, random_state=25, solver='sag')
clf_for_lem.fit(transformed_lem_train, train_ans)
transformed_lem_test = vectorizer.transform(test_lem_data)
print(f1_score(test_ans, clf_for_lem.predict(transformed_lem_test)))

0.9185403114287484


Что-то она тоже особенно ни к чему не привела.

## Подведение итогов

Поигравшись с различными параметрами tfidf, LogReg, а также попробовав определенные методы предобработки текста, приходим к тому, что лучший результат был достигнут над (1,2)-грамами с выбросом слов, встречающихся менее чем в пяти тренировочных текстах, с параметром регуляризации модели C=1.2 и солвером SAG. Для такого случая score правильно предсказанных ответов тестовой выборки аж более 0.93! Если не очень хочется ждать пока tfidf разберется с (1,2)-грамами, очень даже неплохой результат может быть достигнут также и при обычной юниграмной векторизации с теми же параметрами (полученная оценка гармнонического среднего около 0.92).